In [1]:
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from darkflow.net.build import TFNet
import os
import json

In [2]:
def boxing(original_img, predictions):
    # return a new image with a bounding box over the original image using coordinates in predictions
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)

    return newImage

In [3]:
options = {"model": "cfg/yolo.cfg", 
           "load": "bin/yolo.weights", 
           "threshold": 0.3, 
           "gpu": 0.0,
           "labels": "labels.txt",}

tfnet = TFNet(options)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 1.4229869842529297s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep

In [4]:
DATA_DIR = 'data/'
RESULTS_DIR = 'results/'

for filename in os.listdir(DATA_DIR):
    print(filename)
    src = DATA_DIR + filename
    dest = RESULTS_DIR + 'boxed_image_' + filename
    
    img = cv2.imread(src)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = tfnet.return_predict(img)
    #print(results)

    boxed_image = boxing(img, results)
    
    with open(RESULTS_DIR + 'result_'+filename.split('.png')[0] + '.txt','w') as f:
        json.dump(str(results), f)
    
    #plt.imshow(boxed_image)
    plt.imsave(dest, boxed_image)


8.png
9.png
14.png
15.png
17.png
16.png
12.png
13.png
11.png
10.png
21.png
20.png
22.png
23.png
18.png
24.png
25.png
19.png
4.png
5.png
7.png
6.png
2.png
3.png
1.png
0.png
